Importing my libraries

In [1]:
!pip install transformers
import tarfile
import numpy as np
import cupy as cp
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import torch
import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True



In [2]:
# transformers and the tokenzier for transformers
from transformers import BertTokenizer, BertModel
from transformers import CamembertTokenizer, CamembertModel
import torch.nn as nn
# the french tokenizer

tokenizer_en = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer_fr = CamembertTokenizer.from_pretrained('camembert-base')

# declaring special tokens

bert_en = BertModel.from_pretrained('bert-base-uncased').to('cuda')
bert_fr = CamembertModel.from_pretrained('camembert-base').to('cuda')

# Freeze the BERTs model
for param in bert_en.parameters():
    param.requires_grad = False


# Freeze the BERT model
for param in bert_fr.parameters():
    param.requires_grad = False

def tokenize_and_cut(sentence, tokenizer, max_input_length = 512):
    tokens = tokenizer.tokenize(sentence)
    if len(tokens) > max_input_length: # start and end toks
        # Truncate the tokens
        tokens = tokens[:max_input_length]
    elif len(tokens) < max_input_length:
        # Pad the tokens
        tokens += [tokenizer.pad_token] * (max_input_length - len(tokens))
    return tokens
# the tokenizer function

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [3]:
#Test and tokenzie
import torch
tokens = tokenize_and_cut("Comel is are you?", tokenizer_fr)
print(tokens)
input_ids = tokenizer_fr.convert_tokens_to_ids(tokens)
print(input_ids)
input_tensor = torch.tensor([input_ids]).to('cuda')

embedding = bert_fr(input_tensor)[0]
print(embedding)

['▁Com', 'el', '▁is', '▁are', '▁you', '?', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<p

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


tensor([[[-0.0325,  0.0426, -0.1055,  ...,  0.0102, -0.1094, -0.2229],
         [ 0.0795,  0.1601,  0.0176,  ...,  0.0980,  0.0856, -0.2311],
         [ 0.0867,  0.1532,  0.0213,  ...,  0.0952,  0.0869, -0.2270],
         ...,
         [ 0.0911,  0.1415,  0.0246,  ...,  0.0994,  0.0783, -0.2325],
         [ 0.0911,  0.1415,  0.0246,  ...,  0.0994,  0.0783, -0.2325],
         [ 0.0911,  0.1415,  0.0246,  ...,  0.0994,  0.0783, -0.2325]]],
       device='cuda:0')


#Creating helper functions to process the data

In [4]:
# data loaded from the en and fr files in the project
import torch.nn.functional as F

def load_data(file_path):
    with open(file_path, encoding='utf-8') as file:
        lines = [line.strip().lower() for line in file if line.strip()]
    return lines

def preprocess(lang_sentences, percentages, berts, bert_tokenizers, DEVICE="cuda", max_len=10):
    assert len(berts) == len(bert_tokenizers) == len(percentages), "The lengths of berts, tokenizer, and percentages must be the same."

    for i in range(len(berts)):

      berts[i].to(DEVICE)

    languages_mashed = []
    for idx, sentences in enumerate(zip(*lang_sentences)):
        sentence_embeddings = []
        for lang_index, sentence in enumerate(sentences):
            # Tokenize and cut the sentence
            tokens = tokenize_and_cut(sentence, bert_tokenizers[lang_index], 10)
            input_ids = bert_tokenizers[lang_index].convert_tokens_to_ids(tokens)

            input_tensor = torch.tensor([input_ids]).to(DEVICE)

            embedding = berts[lang_index](input_tensor)[0].detach()
            sentence_embeddings.append(embedding)

        combined_embedding = sum([emb * perc for emb, perc in zip(sentence_embeddings, percentages)])
        languages_mashed.append(combined_embedding.cpu())
        if idx % 500 == 0:
          print("saved")
          np.save('mashed_sentences_fr_en_50_50_w_bert_no_rand.npy', languages_mashed) # saving data so doesnt haev to be loaded again
    return languages_mashed

#PreProccessing and creating data

In [6]:

# Load English and French data
import numpy as np

english_sentences = load_data('europarl-v7.fr-en.en')[:2000]
french_sentences = load_data('europarl-v7.fr-en.fr')[:2000]
print(len(english_sentences))
print(len(french_sentences))
print(english_sentences[1])
print(french_sentences[1])

# **mish mash** with 0.5 and 0.5 percentage points
mashed_sentences = preprocess([english_sentences, french_sentences],
                          [0.5, 0.5],
                          [bert_en, bert_fr],
                          [tokenizer_en, tokenizer_fr],
                          "cuda")

print(mashed_sentences[0])
np.save('mashed_sentences_fr_en_50_50_w_bert_no_rand.npy', mashed_sentences) # saving data so doesnt haev to be loaded again

2000
2000
i declare resumed the session of the european parliament adjourned on friday 17 december 1999, and i would like once again to wish you a happy new year in the hope that you enjoyed a pleasant festive period.
je déclare reprise la session du parlement européen qui avait été interrompue le vendredi 17 décembre dernier et je vous renouvelle tous mes vux en espérant que vous avez passé de bonnes vacances.
saved
saved
saved
saved
tensor([[[-1.4093e-02,  5.4337e-03,  1.2184e-01,  ..., -2.3212e-01,
          -9.1160e-02,  5.5050e-02],
         [-5.8312e-02, -1.4492e-02,  3.0182e-02,  ..., -3.5545e-01,
          -1.0365e-01,  7.8641e-02],
         [-7.6053e-02,  3.6148e-02, -2.1973e-03,  ..., -3.6919e-01,
          -6.2693e-02,  7.5104e-02],
         ...,
         [ 3.2398e-02,  3.1216e-01,  2.4066e-01,  ..., -3.0676e-01,
           1.5297e-02,  1.1020e-02],
         [ 9.1267e-02,  3.1950e-01,  2.6753e-01,  ..., -2.7500e-01,
           2.8785e-04,  3.3211e-02],
         [ 9.9911e-02,

#Here after processing the data it can be loaded with pytorch data loader object

In [9]:
from torch.utils.data import Dataset, DataLoader

class MashedDataset(Dataset):
  def __init__(self, data):
        self.data = data

  def __len__(self):
      return len(self.data)

  def __getitem__(self, idx):
      return self.data[idx]

mashed_sentences = np.load('mashed_sentences_fr_en_50_50_w_bert_no_rand.npy')[:2000]
print(mashed_sentences.shape)

#mashed_sentences = mashed_sentences[:]
# only using first 100
print(mashed_sentences.shape)

mashed_sentences_dataset = MashedDataset(torch.from_numpy(mashed_sentences))
# make sure to from numpy it

batch_size = 128
mashed_sentences_data_loader = DataLoader(mashed_sentences_dataset,
                                   batch_size=batch_size,
                                   shuffle=True,
                                    drop_last=True)

(2000, 1, 10, 768)
(2000, 1, 10, 768)


#Here is the onehot outputs for characters and their encodings

In [10]:
one_hot_characters = ['a', 'à','â', 'æ', 'b', 'c', 'ç', 'd', 'e', 'é', 'è', 'ê',
                      'ë', 'œ', 'f', 'g', 'h', 'i', 'î', 'ï', 'j', 'k', 'l',
                      'm', 'n', 'o', 'ô', 'p', 'q', 'r', 's', 't', 'u', 'ù','û',
                      'ü', 'v', 'w', 'x', 'y', 'ÿ' 'z', "'", ' ', ' ', ' ']

                      # space character and ' included forcing more spaces
# this is both english and french characters discluding the overlap
# capitals are **banned** and arent used
# helper dictionaries for conversions
char_to_index = {char: index for index, char in enumerate(one_hot_characters)}
index_to_char = {index: char for index, char in enumerate(one_hot_characters)}


#Generator Architecture

The way this works is it will take in some length vector and then from it it will create the one hot matrix which represents the generated mish mashed sentence


In [11]:
class Generator(nn.Module):
    def __init__(self, input_size=100, seq_length=60):
        # since only at max 10 words maybe 60 characters can be outputted max
        super(Generator, self).__init__()
        self.seq_length = seq_length
        self.gru = nn.GRU(input_size, 256, num_layers=1, batch_first=True)
        self.relu = nn.ReLU()
        self.linear = nn.Linear(256, len(one_hot_characters))
        self.softmax = nn.Softmax(dim=2)

        # Initialize weights
        self.init_weights()

    def init_weights(self):
        for m in self.modules():
            if isinstance(m, nn.GRU):
                nn.init.xavier_uniform_(m.weight_ih_l0)
                nn.init.xavier_uniform_(m.weight_hh_l0)
            elif isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)

    def forward(self, x, temperature=1.0):
    # Repeat the input noise vector seq_length times to create a sequence
      x = x.repeat(1, self.seq_length, 1)
      out, _ = self.gru(x)  # Only take the output, ignore the hidden state
      out = self.relu(out)
      out = self.linear(out)
      out = out / temperature  # Apply the temperature parameter
      out = self.softmax(out)
      return out


#Call the generator to see what it outputs untrained

In [17]:
def generate_sequences(generator, noise, DEVICE='cuda'):
    # Initialize an empty list to store the sequences
    sequences = []
    # Iterate over the batch dimension of the noise tensor
    for i in range(noise.size(0)):
        # Generate a sequence of one-hot vectors for each noise vector
        one_hot_sequence = generator(noise[i].unsqueeze(0).to(DEVICE), 1)
        # Convert the one-hot vectors to character indices
        char_indices = torch.argmax(one_hot_sequence, dim=2)
        # Convert the character indices to characters
        sequence = ''.join(index_to_char[index.item()] for index in char_indices[0])
        sequences.append(sequence)
    return sequences

# this is just for testing not important
noise = torch.randn(2, 100)
generator = Generator().to('cuda')
sequences = generate_sequences(generator, noise)

print(sequences)

# ill embed this into english and then french and combine it to see what should
# happen from it

def sequence_to_mash_embed(sequences, percentages, berts, bert_tokenizers, DEVICE="cuda"):
    # Initialize an empty list to store the embeddings
    lang_embeddings = [] # this should be a tensor of the
    # num langs * size batch_size * 10 * 768
    tokenized_and_id_sentences = []

    for lang_index in range(len((berts))):
        sentences_ids = []
        for sequence in sequences:
           tokens = tokenize_and_cut(sequence, bert_tokenizers[lang_index], 10)
           input_ids = bert_tokenizers[lang_index].convert_tokens_to_ids(tokens)
           sentences_ids.append(input_ids)
        tokenized_and_id_sentences.append(sentences_ids)

    for i in range(len(tokenized_and_id_sentences)):
        # Tokenize and cut the sentence
        input_tensor = torch.tensor(tokenized_and_id_sentences[i]).to(DEVICE)
        embedding = berts[i](input_tensor)[0].detach()
        lang_embeddings.append(embedding)

    mashed_embedding = torch.zeros_like(lang_embeddings[0])
    for embedding, percent in (lang_embeddings, percentages):
        mashed_embedding += embedding * percent
    return mashed_embedding


bert_en.to('cuda')
bert_fr.to('cuda')

mashed_embedding = sequence_to_mash_embed(sequences, [0.5, 0.5],
                          [bert_en, bert_fr],
                          [tokenizer_en, tokenizer_fr],
                          "cuda")
print(mashed_embedding) # this might work
# otherwise some other embedding scheme needs to be defined for the mashed language


['uuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuu', '                                                            ']
tensor([[[0.2588, 0.2478, 0.2985,  ..., 0.2496, 0.2545, 0.2104],
         [0.2584, 0.2395, 0.0453,  ..., 0.2895, 0.2518, 0.1752],
         [0.2764, 0.2203, 0.1646,  ..., 0.3032, 0.2504, 0.1375],
         ...,
         [0.2800, 0.2378, 0.1968,  ..., 0.2887, 0.2506, 0.1868],
         [0.2858, 0.2417, 0.1976,  ..., 0.2836, 0.2501, 0.1799],
         [0.2661, 0.2400, 0.2261,  ..., 0.2767, 0.2488, 0.1764]],

        [[0.2505, 0.2531, 0.2818,  ..., 0.2467, 0.2280, 0.2416],
         [0.2691, 0.2483, 0.2790,  ..., 0.2456, 0.2265, 0.2403],
         [0.2723, 0.2446, 0.2806,  ..., 0.2455, 0.2275, 0.2401],
         ...,
         [0.2660, 0.2511, 0.2699,  ..., 0.2450, 0.2268, 0.2397],
         [0.2666, 0.2505, 0.2692,  ..., 0.2451, 0.2270, 0.2396],
         [0.2665, 0.2502, 0.2709,  ..., 0.2454, 0.2277, 0.2389]]],
       device='cuda:0')


#Discriminator Architecture

This has to figure out if something is real or fake

In [18]:
class Discriminator(nn.Module):
    def __init__(self, input_size=768, hidden_size=128, dropout=0.3):
        super(Discriminator, self).__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers=1, batch_first=True)
        self.relu = nn.ReLU()
        self.linear = nn.Linear(hidden_size, 1)
        self.dropout = nn.Dropout(dropout)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out, _ = self.gru(x)  # Only take the output, ignore the hidden state
        out = self.relu(out)
        out = self.linear(out[:, -1, :])  # Only take the last output of the sequence
        out = self.dropout(out)
        out = self.sigmoid(out)
        return out

#Testing the untrained discriminator on the previous embeddings

In [23]:
# Convert the numpy array to a PyTorch tensor
print(mashed_embedding.shape)
mashed_embedding_tensor = mashed_embedding

# Pass the tensor to the discriminator
discriminator = Discriminator().to('cuda')
output = discriminator(mashed_embedding_tensor)

# Pass the embeddings through the discriminator
print(output)


torch.Size([128, 10, 768])
tensor([[0.5000],
        [0.5000],
        [0.4682],
        [0.5000],
        [0.4752],
        [0.4664],
        [0.4906],
        [0.4646],
        [0.5000],
        [0.5000],
        [0.4916],
        [0.5000],
        [0.4854],
        [0.4752],
        [0.5000],
        [0.4912],
        [0.4695],
        [0.4794],
        [0.4782],
        [0.4751],
        [0.4688],
        [0.4916],
        [0.5000],
        [0.4904],
        [0.4639],
        [0.5000],
        [0.5000],
        [0.4688],
        [0.4695],
        [0.4639],
        [0.4803],
        [0.4613],
        [0.4821],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.4655],
        [0.5000],
        [0.4886],
        [0.4828],
        [0.5000],
        [0.4902],
        [0.4481],
        [0.4588],
        [0.5000],
        [0.5000],
        [0.4821],
        [0.5000],
        [0.5000],
        [0.5000],
        [0.4782],
        [0.4804],
        [0.4627],
 

#Training Setup

In [24]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)
D = Discriminator().to(DEVICE)
G = Generator(seq_length=32).to(DEVICE)
# shoving hopefully everything to the gpu

max_epoch = 50 # going for 50 epochs
step = 0 # step through the data
n_noise = 100 # size of noise vector

criterion = nn.BCELoss()
D_opt = torch.optim.Adam(D.parameters(), lr=10e-4, betas=(0.5, 0.999))
G_opt = torch.optim.Adam(G.parameters(), lr=10e-4, betas=(0.5, 0.999))
# optimizers for both the discriminator and generator alongside a
# binary cross entropy loss

# We will denote real images as 1s and fake images as 0s
# This is why we needed to drop the last batch of the data loader
D_labels = torch.ones([batch_size, 1]).to(DEVICE) # Discriminator label: real
D_fakes = torch.zeros([batch_size, 1]).to(DEVICE) # Discriminator Label: fake

cuda


#Training Loop

In [ ]:
import matplotlib.pyplot as plt
import time
# import pyplot to plot images
#from google.colab import drive
#drive.mount('/content/gdrive')

start_time = time.time()
for epoch in range(max_epoch):
    for idx, word_embeddings in enumerate(mashed_sentences_data_loader):
        # Training Discriminator
        x = word_embeddings.float().to(DEVICE)
        # rehsaping x to have the x dimension in it for the vectores
        x = x.squeeze() # removing the bad dims
        x = x.view(batch_size, 10, 768)  # adding in the 10 size
        x_outputs = D(x) # input includes labels
        D_x_loss = criterion(x_outputs, D_labels) # Discriminator loss for real images

        z = torch.randn(batch_size, n_noise).to(DEVICE)

        # the generator outputs a sequence that sequence must then be converted into
        # embeddings that are passsed to the dsicriminator
         # the generator and the randomness to make a sequence
        sequences = generate_sequences(G, z)
        mashed_embedding = sequence_to_mash_embed(
                          sequences, [0.5, 0.5],
                          [bert_en, bert_fr],
                          [tokenizer_en, tokenizer_fr],
                          "cuda")
        mashed_embedding_tensor = mashed_embedding_tensor.squeeze()
        z_outputs = D(mashed_embedding_tensor) # input to both generator and discriminator includes labels
        D_z_loss = criterion(z_outputs, D_fakes) # Discriminator loss for fake images
        D_loss = D_x_loss + D_z_loss # Total Discriminator loss

        D.zero_grad()
        D_loss.backward()
        D_opt.step()
        # updating the discriminator model

        # Training Generator
        z = torch.randn(batch_size, n_noise).to(DEVICE) # creating the random vector alongside the batch proper
        train_sequences = generate_sequences(G, z)
        mashed_embedding_train = sequence_to_mash_embed(
                          train_sequences, [0.5, 0.5],
                          [bert_en, bert_fr],
                          [tokenizer_en, tokenizer_fr],
                          "cuda")
        mashed_embedding_tensor_train = mashed_embedding_tensor_train.squeeze()

        z_outputs = D(mashed_embedding_tensor_train)
        G_loss = -1 * criterion(z_outputs, D_fakes) # Generator loss is negative disciminator loss

        G.zero_grad()
        G_loss.backward()
        G_opt.step()
        # updating the generator model

        if step % 500 == 0:
            print('Epoch: {}/{}, Step: {}, D Loss: {}, G Loss: {} time: '.format(epoch, max_epoch, step, D_loss.item(), G_loss.item(), time.time() - start_time))
            # done to view teh loss
        step += 1

    if epoch+1 in [1, 5, 10, 15, 20, 25, 30, 50]:
      # if in the 1st (done for making sure everything is good)
      # or the 10th or 30th or 50th epoch then display what the
      # generator has so far
      model_save_name = 'discriminator_bert.pt'
      path = f"{model_save_name}"
      torch.save(D.state_dict(), path)

      model_save_name = 'generator_bert.pt'
      path = f"{model_save_name}"
      torch.save(G.state_dict(), path)

      print(f"on epoch {epoch + 1}")
      noise = torch.randn(1, 100).to(DEVICE)
      G.eval()  # eval mode
      sequences = generate_sequences(generator, noise)
      print(sequences)
      # show the plot from get sample images
      G.train()
      # back to trianing the genrator



Epoch: 0/50, Step: 0, D Loss: 1.2801696062088013, G Loss: -0.7939716577529907 time: 
on epoch 1
['ÿzÿzœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœœ']
